In [ ]:
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import cv2

# Haar Cascade 모델 다운로드 및 로드
!wget -O haarcascade_frontalface_default.xml https://github.com/opencv/opencv/raw/4.x/data/haarcascades/haarcascade_frontalface_default.xml
!wget -O haarcascade_eye.xml https://github.com/opencv/opencv/raw/4.x/data/haarcascades/haarcascade_eye.xml

# Haar Cascade 모델 로드
face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('./haarcascade_eye.xml')


--2024-12-12 02:00:40--  https://github.com/opencv/opencv/raw/4.x/data/haarcascades/haarcascade_frontalface_default.xml
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/opencv/opencv/4.x/data/haarcascades/haarcascade_frontalface_default.xml [following]
--2024-12-12 02:00:40--  https://raw.githubusercontent.com/opencv/opencv/4.x/data/haarcascades/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘haarcascade_frontalface_default.xml’

haarcascade_frontal 100%[===================>] 908.33K  --.-KB/s    in 0.02s   

2024-12-12 02

In [ ]:
def extract_eyes(image_path, size=(64, 64)):
    img = cv2.imread(image_path)  # 이미지 로드
    if img is None:
        raise ValueError(f"이미지를 로드할 수 없습니다: {image_path}")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 그레이스케일 변환

    # 얼굴 검출
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    eyes = []

    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]  # 얼굴 영역 추출

        # 눈 검출
        detected_eyes = eye_cascade.detectMultiScale(face_roi)
        for (ex, ey, ew, eh) in detected_eyes:
            eye_roi = face_roi[ey:ey+eh, ex:ex+ew]  # 눈 영역 추출
            eye_resized = cv2.resize(eye_roi, size)  # 크기 조정
            eyes.append(eye_resized)

    return eyes  # 검출된 눈 목록 반환


In [ ]:
import os
import numpy as np

def preprocess_images(folder, label, size=(64, 64)):
    data = []
    labels = []
    for file_name in os.listdir(folder):
        file_path = os.path.join(folder, file_name)
        try:
            eyes = extract_eyes(file_path, size)  # 눈 추출
            for eye in eyes:
                data.append(eye.flatten())  # 플래튼 (1D 배열로 변환)
                labels.append(label)  # 라벨 추가
        except Exception as e:
            print(f"이미지 처리 실패: {file_name}, 에러: {e}")
    return np.array(data), np.array(labels)


In [ ]:
from sklearn.model_selection import train_test_split

# Google Drive 데이터 경로
base_path = '/content/drive/MyDrive/ColabNotebooks/eye_image'
double_folder = os.path.join(base_path, 'double')  # 유쌍 폴더
single_folder = os.path.join(base_path, 'single')  # 무쌍 폴더

# 유쌍과 무쌍 데이터 전처리
double_data, double_labels = preprocess_images(double_folder, label=1)  # 유쌍: 라벨 1
single_data, single_labels = preprocess_images(single_folder, label=0)  # 무쌍: 라벨 0

# 데이터 병합
X = np.concatenate((double_data, single_data), axis=0)
y = np.concatenate((double_labels, single_labels), axis=0)
print(f"총 데이터 크기: {X.shape}, 레이블 크기: {y.shape}")

# 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


총 데이터 크기: (55, 4096), 레이블 크기: (55,)


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# SVM 모델 학습
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train, y_train)

# 예측 및 평가
y_pred = svm_model.predict(X_test)
print(f"정확도: {accuracy_score(y_test, y_pred):.2f}")
print(classification_report(y_test, y_pred))


정확도: 0.55
              precision    recall  f1-score   support

           0       0.62      0.71      0.67         7
           1       0.33      0.25      0.29         4

    accuracy                           0.55        11
   macro avg       0.48      0.48      0.48        11
weighted avg       0.52      0.55      0.53        11



In [ ]:
from collections import Counter

# 다수결 방식으로 유쌍/무쌍 예측
def predict_eye_type_majority(model, image_path, size=(64, 64)):
    eyes = extract_eyes(image_path, size)
    predictions = []
    for eye in eyes:
        eye_flat = eye.flatten().reshape(1, -1)  # 1D로 변환 후 reshape
        pred = model.predict(eye_flat)
        predictions.append(pred[0])  # 0: 무쌍, 1: 유쌍

    if predictions:  # 눈이 검출된 경우
        counts = Counter(predictions)  # 예측 결과 빈도 계산
        majority = counts.most_common(1)[0][0]  # 가장 빈도가 높은 라벨
        return "유쌍" if majority == 1 else "무쌍"
    else:  # 눈이 검출되지 않은 경우
        return "눈이 검출되지 않았습니다"
# 테스트 이미지 경로
test_image_path = '/content/drive/MyDrive/ColabNotebooks/eye_image/test_image/3.jpg'

try:
    result_majority = predict_eye_type_majority(svm_model, test_image_path)
    print(f"예측 결과 (다수결): {result_majority}")
except Exception as e:
    print(f"테스트 이미지 처리 실패: {e}")


예측 결과 (다수결): 무쌍


In [ ]:
def predict_eye_type(model, image_path, size=(64, 64)):
    eyes = extract_eyes(image_path, size)
    predictions = []
    for eye in eyes:
        eye_flat = eye.flatten().reshape(1, -1)  # 1D로 변환 후 reshape
        pred = model.predict(eye_flat)
        predictions.append("유쌍" if pred[0] == 1 else "무쌍")
    return predictions

# 테스트 이미지 예측
test_image_path = '/content/drive/MyDrive/ColabNotebooks/eye_image/test_image/3.jpg'  # 테스트 이미지 경로
try:
    results = predict_eye_type(svm_model, test_image_path)
    print(f"예측 결과: {results}")
except Exception as e:
    print(f"테스트 이미지 처리 실패: {e}")


예측 결과: ['무쌍', '무쌍', '유쌍', '무쌍', '유쌍', '유쌍']
